# Advanced Modeling and Simulation Course
### Instructor: Dr. M. Nedim ALPDEMİR
The first part of the exercises contains examples of Discrete Event Simulation.
For this part we use an open source Discrete Event Simulation package written in Python called __SimPy__

## Obtaining SimPy
[SimPy is available from https://pypi.org/project/simpy/](https://simpy.readthedocs.io/en/latest/contents.html)


The examples in this part start with a relatively simple problem and increase in complexity as we move forward.

## Brief introduction to SimPy Basic Concepts
* The behavior of active components (like vehicles, customers or messages) is modeled with ___processes___. 
* All processes live in an ___environment___. 
* They interact with the environment and with each other via ___events___.

### Processes
* Processes are described by simple Python **generators**. 
* You can call them process function or process method, depending on whether it is a normal function or method of a class. 
* During their lifetime, they create **events** and **yield** them in order to wait for them to be **triggered**.

### Events
* When a process yields an event, the process gets suspended. 
* SimPy resumes the process, when the event occurs (we say that the event is triggered). 
* Multiple processes can wait for the same event. SimPy resumes them in the same order in which they yielded that event.

### Time Management
* An important event type is the **Timeout**. 
* Events of this type are triggered after a certain amount of (simulated) time has passed. 
* They allow a process to sleep (or hold its state) for the given time. A Timeout and all other events can be created by calling the appropriate method of the Environment that the process lives in (Environment.timeout() for example).





## Module 1

The first exercise is about a simple car that keeps driving for 20 minutes and then parks in 5 minutes


In [1]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 20
        yield env.timeout(trip_duration)

import simpy
env = simpy.Environment()
env.process(car(env))
env.run(until=100)

Start parking at 0
Start driving at 5
Start parking at 25
Start driving at 30
Start parking at 50
Start driving at 55
Start parking at 75
Start driving at 80


# Gas Station Refueling example

###  Covers the following concepts:

- Resources: Resource
- Resources: Container
- Waiting for other processes

### Problem Description:
  * A gas station has a limited number of gas pumps that share a common fuel reservoir. Cars randomly arrive at the gas station, request one of the fuel pumps and start refueling from that reservoir.

  * A gas station control process observes the gas station's fuel level and calls a tank truck for refueling if the station's level drops below a threshold.
  
### We define the following 

  * **Fuel Pump** : a Container
  * **Car** : a simulation Entity
  * **Gas Station Control** : a Process
  * **Car generator** :  a Process
  * **Gas Station** :  a Resource
  

In [2]:
import itertools
import random

import simpy


RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds

In [3]:
def car(name, env, gas_station, fuel_pump):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of gas from it. If the stations reservoir is
    depleted, the car has to wait for the tank truck to arrive.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s arriving at gas station at %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Request one of the gas pumps
        yield req
        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        #print('%s requesting %.1f litres of gas' % (name, liters_required))
        yield fuel_pump.get(liters_required)
        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s finished refueling in %.1f seconds.' % (name,
                                                          env.now - start))


In [4]:
def gas_station_control(env, fuel_pump):
    """Periodically check the level of the *fuel_pump* and call the tank
    truck if the level falls below a threshold."""
    while True:
        #print('pump level %d' % fuel_pump.level)
        if fuel_pump.level * 100 / fuel_pump.capacity  < THRESHOLD:
            # We need to call the tank truck now!
            print('Calling tank truck at %d' % env.now)
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds

In [5]:
def tank_truck(env, fuel_pump):
    """Arrives at the gas station after a certain delay and refuels it."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Tank truck arriving at time %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Tank truck refuelling %.1f liters.' % ammount)
    yield fuel_pump.put(ammount)


In [6]:
def car_generator(env, gas_station, fuel_pump):
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, gas_station, fuel_pump))

In [7]:
# Setup and start the simulation
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

Gas Station refuelling
Car 0 arriving at gas station at 87.0
Car 0 finished refueling in 18.5 seconds.
Car 1 arriving at gas station at 129.0
Car 1 finished refueling in 19.0 seconds.
Car 2 arriving at gas station at 284.0
Car 2 finished refueling in 21.0 seconds.
Car 3 arriving at gas station at 385.0
Car 3 finished refueling in 13.5 seconds.
Car 4 arriving at gas station at 459.0
Calling tank truck at 460
Car 4 finished refueling in 22.0 seconds.
Car 5 arriving at gas station at 705.0
Car 6 arriving at gas station at 750.0
Tank truck arriving at time 760
Tank truck refuelling 188.0 liters.
Car 6 finished refueling in 29.0 seconds.
Car 5 finished refueling in 76.5 seconds.
Car 7 arriving at gas station at 891.0
Car 7 finished refueling in 13.0 seconds.
